In [2]:
import sounddevice as sd
import numpy as np
import librosa
from keras.models import load_model
from sklearn.preprocessing import LabelEncoder, StandardScaler
import joblib
model = load_model('my_model.keras')
scaler = joblib.load('scaler.save') 
encoder = LabelEncoder()
encoder.classes_ = np.load('classes.npy', allow_pickle=True)
duration = 5 # seconds
sample_rate = 22050 
def extract_features(audio, sr=22050, mfcc=True, chroma=True, mel=True):
    result = np.array([])
    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=20).T, axis=0)
        result = np.hstack((result, mfccs))
    if chroma:
        stft = np.abs(librosa.stft(audio))
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sr).T, axis=0)
        result = np.hstack((result, chroma))
    if mel:
        mel = np.mean(librosa.feature.melspectrogram(y=audio, sr=sr).T, axis=0)
        result = np.hstack((result, mel))
    return result
def predict_audio():
    print("Recording audio...")
    audio = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='float32')
    sd.wait()  # Wait until recording is finished
    print("Audio recording complete.")
    
    audio = audio.flatten()  # Flatten the audio array
    features = extract_features(audio, sr=sample_rate)
    features = features.reshape(1, -1)
    features = scaler.transform(features)
    
    prediction = model.predict(features)
    predicted_label = np.argmax(prediction, axis=1)
    label = encoder.inverse_transform(predicted_label)
    
    if label[0] == 'cry':
        print("little one is up")
    else:
        print("Non cry")

In [15]:
predict_audio()

Recording audio...
Audio recording complete.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Non cry
